In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
df = pd.read_csv("training_dataset1.csv")

In [3]:
X = df[['Num_escuelas',
 'Num_hospitales',
 'Num_iglesias',
 'Num_zonas_tipicas',
   ]]

y = df["Num_viajes"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [5]:
linear_model = LinearRegression()
linear_model.fit(X, y)

LinearRegression()

In [6]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np 

yfit_linear = pd.DataFrame(linear_model.predict(X_train), index = X_train.index, columns = ["Num_viajes"]).clip(0.)

ypred_linear = pd.DataFrame(linear_model.predict(X_test), index = X_test.index, columns = ["Num_viajes"]).clip(0.)


print('Linear RMSLE :', np.sqrt(msle(y_test, ypred_linear)))
print('Linear MAE :', mae(y_test, ypred_linear))


Linear RMSLE : 0.693876749693628
Linear MAE : 19388.925108526328


In [7]:
y_train_linear = linear_model.predict(X_train)

print('XGBoost mae :', mae(y_train, y_train_linear))


XGBoost mae : 16385.277101314692


# Evaluación del modelo

Para evaluar el modelo, revisaremos el % de error utilizando MAE

### Training error

In [8]:
dataframe = pd.DataFrame()
dataframe['Num_viajes'] = y_train
dataframe['Num_viajes_p'] = yfit_linear
dataframe['dif'] = abs(dataframe['Num_viajes'] - dataframe['Num_viajes_p'])
import math
dataframe['Num_viajes_p'] =  dataframe['Num_viajes_p'].apply(lambda x: math.ceil(x))
dataframe["Dif_perc"] =  dataframe["dif"] / dataframe["Num_viajes"] *100
print("The % of error in training is", dataframe["Dif_perc"].mean())


The % of error in training is 31.47445253338586


### Testing error

In [9]:
dataframe = pd.DataFrame()
dataframe['Num_viajes'] = y_test
dataframe['Num_viajes_p'] = ypred_linear
dataframe['dif'] = abs(dataframe['Num_viajes'] - dataframe['Num_viajes_p'])
import math
dataframe['Num_viajes_p'] =  dataframe['Num_viajes_p'].apply(lambda x: math.ceil(x))
dataframe["Dif_perc"] =  dataframe["dif"] / dataframe["Num_viajes"] *100
print("The % of error in test is", dataframe["Dif_perc"].mean())


The % of error in test is 67.03080147997044


## Interpretación

In [13]:
import statsmodels.api as sm
X = sm.add_constant(X_train)
model = sm.OLS(y_train,X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             Num_viajes   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                     29.17
Date:                Mon, 28 Nov 2022   Prob (F-statistic):           5.45e-11
Time:                        20:12:52   Log-Likelihood:                -480.40
No. Observations:                  42   AIC:                             970.8
Df Residuals:                      37   BIC:                             979.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              1.116e+04   6602.80

    Al analizar los resultados de la regresión lineal podemos ver que el impacto que hace el intercepto (la constante) no es estadísticamente significativo al tener un P value de 0.09, superando el margen de tolerancia de 0.05. 

    Asímismo vemos que el número de escuelas es la variable que tiene una mayor significancia estadística, ya que su t value es mayor al de las demás variables. Este número tiene una magnitud pequeña, y se relaciona de manera positiva con el número de viajes recibido. Es decir que a mayor escuelas, mayor cantidad de viajes recibidos en una comuna. 

    Con respecto a los hospitales, vemos que es estadísticamente significativo (p value menor a 0.05), además de tener una magnitud mediana. Esta variable se relaciona positivamente con la cantidad de viajes recibidos en una comuna, es decir que a mayor cantidad de hospitales, mayor cantidad de viajes recibidos. 

    Hablando de las iglesias, esta variable es de una magnitud alta y es estadísticamente significativa. Se relaciona positivamente con los viajes recibidos por comuna, por lo que a mayor cantidad de iglesias, más viajes se reciben en una comuna.
    
    En lo que respecta a las zonas típicas, esta variable es de una magnitud alta y estadísticamente significativa. Tiene una relación negativa con la cantidad de viajes, así que mientras más zonas típicas se tengan, menos viajes se recibe en la comuna.